In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

# Import the cross-validation tool
from sklearn.model_selection import cross_val_score

In [4]:
# Load the data, keeping the features (X) and target (y) separated
iris = load_iris(as_frame=True)
x, y = iris.data, iris.target

print(f"Total samples: {len(x)}")

Total samples: 150


In [5]:
# Instantiate the model
logreg = LogisticRegression(max_iter=200)

print("Logistic Regression model instantiated.")

Logistic Regression model instantiated.


In [6]:
# Perform 5-Fold Cross-Validation (CV)
# cv=5 means 5 folds. The scoring='accuracy' tells it what metric to return.
cv_scores = cross_val_score(
    estimator=logreg, # The model to use
    X=x,              # The FULL feature set (CV handles the splitting)
    y=y,              # The FULL target set
    cv=5,             # Number of folds
    scoring='accuracy'
)

print("Individual Cross-Validation Scores (5 Folds):")
print(cv_scores)

Individual Cross-Validation Scores (5 Folds):
[0.96666667 1.         0.93333333 0.96666667 1.        ]


In [7]:
# Calculate the mean (average performance)
mean_cv_score = np.mean(cv_scores)

# Calculate the standard deviation (measure of variability/robustness)
std_cv_score = np.std(cv_scores)

print(f"\nMean CV Accuracy: {mean_cv_score:.4f}")
print(f"Standard Deviation of CV Scores: {std_cv_score:.4f}")


Mean CV Accuracy: 0.9733
Standard Deviation of CV Scores: 0.0249


In [8]:
## Mean accuracy = High, therefore model is relatively accurate
## Std Dev = Low, therefore model is precise i.e. repeatable

In [9]:
# Import the remaining classifiers needed for comparison
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier # Adding KNN back for full comparison

# Instantiate all the models you want to test
models = [
    ('Logistic Regression', LogisticRegression(max_iter=200, random_state=42)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('SVC', SVC(random_state=42))
]

# Initialize a list to store the results
cv_results = []

In [10]:
# Perform 5-Fold CV for each model
for name, model in models:
    # Use cross_val_score
    cv_scores = cross_val_score(
        estimator = model,
        X = x,
        y = y,
        cv = 5,
        scoring='accuracy'
    )
    # Store the avg score and std.dev
    cv_results.append({
        'Model': name,
        'Mean CV Accuracy': np.mean(cv_scores),
        'STD': np.std(cv_scores)
    })

    print(f"Completed CV for: {name}")

print("\nAll CV runs complete.")


Completed CV for: Logistic Regression
Completed CV for: KNN
Completed CV for: Random Forest
Completed CV for: SVC

All CV runs complete.


In [11]:
# Create the comparison table
cv_summary_df = pd.DataFrame(cv_results)

# Format the results for better readability and sort by accuracy
cv_summary_df['Mean CV Accuracy'] = cv_summary_df['Mean CV Accuracy'].map('{:.4f}'.format)
cv_summary_df['STD'] = cv_summary_df['STD'].map('{:.4f}'.format)
cv_summary_df = cv_summary_df.sort_values(by='Mean CV Accuracy', ascending=False)

print("--- Cross-Validation Model Comparison ---")
print(cv_summary_df)

--- Cross-Validation Model Comparison ---
                 Model Mean CV Accuracy     STD
0  Logistic Regression           0.9733  0.0249
1                  KNN           0.9667  0.0211
2        Random Forest           0.9667  0.0211
3                  SVC           0.9667  0.0211


In [12]:
# Import the tuning tool
from sklearn.model_selection import GridSearchCV

# Import the model we are tuning (SVC)
from sklearn.svm import SVC

In [15]:
# Define the parameter grid (all combinations of these values will be tested)
param_grid = {
    # Test C values
    'C' : [0.1, 1, 10, 100],
    # Test gamma values
    'gamma': [1, 0.1, 0.01, 0.001],
    # We will only use the Radial Basis Function (RBF) kernel
    'kernel': ['rbf']
}

print("Parameter grid defined. testing 4 * 4 * 1 = 16 combinations.")

Parameter grid defined. testing 4 * 4 * 1 = 16 combinations.


In [17]:
# Instantiate the SVC model
svc_base = SVC(random_state=42)

#Instantiate GridSearchCV
grid_search = GridSearchCV(
    estimator = svc_base,
    param_grid = param_grid,
    scoring = 'accuracy', # Metric to optimize
    cv = 5,              # Use 5-Fold Cross-Validation
    verbose = 3,         # Shows progress output during the long run
    n_jobs = -1          # Use all available CPU cores
)

In [18]:
grid_search.fit(x,y)

print("\nGrid Search Completed.")

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Grid Search Completed.


In [19]:
# Get highest score
best_score = grid_search.best_score_
print(f"Best Mean CV Accuracy found: {best_score:.4f}")

# Get the combination of settings with highest score
best_params = grid_search.best_params_
print("Best Hyperparameters:")
print(best_params)

# Store the final, best-tuned model
best_model = grid_search.best_estimator_

Best Mean CV Accuracy found: 0.9800
Best Hyperparameters:
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
